# وكلاء Azure AI مع دعم بروتوكول سياق النموذج (MCP) - بايثون

يستعرض هذا الدفتر كيفية استخدام وكلاء Azure AI مع أدوات بروتوكول سياق النموذج (MCP) في بايثون. يوضح كيفية إنشاء وكيل ذكي يمكنه الاستفادة من خوادم MCP الخارجية (مثل Microsoft Learn) للحصول على قدرات محسّنة باستخدام المصادقة بدون مفاتيح.


## تثبيت حزم بايثون المطلوبة

أولاً، نحتاج إلى تثبيت حزم بايثون الضرورية:
- **azure-ai-projects**: حزمة SDK الأساسية لمشاريع Azure AI
- **azure-ai-agents**: حزمة SDK لوكلاء Azure AI لإنشاء وإدارة الوكلاء
- **azure-identity**: توفر مصادقة بدون مفاتيح باستخدام DefaultAzureCredential
- **mcp**: تنفيذ بروتوكول Model Context لـ Python


## فوائد المصادقة بدون مفاتيح

يوضح هذا الدفتر فوائد **المصادقة بدون مفاتيح** التي تقدم العديد من المزايا:
- ✅ **لا حاجة لإدارة مفاتيح API** - تستخدم المصادقة المستندة إلى هوية Azure
- ✅ **أمان معزز** - لا يتم تخزين أي أسرار في الكود أو ملفات التكوين
- ✅ **تدوير تلقائي للاعتمادات** - تتولى Azure إدارة دورة حياة الاعتمادات
- ✅ **التحكم في الوصول المستند إلى الأدوار** - يستخدم Azure RBAC لتحديد الأذونات بدقة
- ✅ **دعم متعدد البيئات** - يعمل بسلاسة عبر بيئات التطوير والإنتاج

يقوم `DefaultAzureCredential` باختيار أفضل مصدر متاح للاعتمادات تلقائيًا:
1. **الهوية المُدارة** (عند التشغيل في Azure)
2. **اعتمادات Azure CLI** (أثناء التطوير المحلي)
3. **اعتمادات Visual Studio**
4. **متغيرات البيئة** (إذا تم تكوينها)
5. **المصادقة عبر المتصفح التفاعلي** (كخيار احتياطي)


## إعداد المصادقة بدون مفاتيح

**المتطلبات الأساسية للمصادقة بدون مفاتيح:**

### للتطوير المحلي:
```bash
# Install Azure CLI and login
az login
# Verify your identity
az account show
```

### لبيئات Azure:
- تفعيل **الهوية المُدارة المُعينة من النظام** على مورد Azure الخاص بك
- تعيين الأدوار المناسبة في **RBAC** للهوية المُدارة:
  - `Cognitive Services OpenAI User` للوصول إلى Azure OpenAI
  - `AI Developer` للوصول إلى مشاريع Azure AI

### متغيرات البيئة (اختياري):
```python
# These are automatically detected by DefaultAzureCredential
# AZURE_CLIENT_ID=<your-client-id>
# AZURE_CLIENT_SECRET=<your-client-secret>
# AZURE_TENANT_ID=<your-tenant-id>
```

**لا حاجة لمفاتيح API أو سلاسل اتصال!** 🔐


In [ ]:
! pip install azure-ai-projects -U
! pip install azure-ai-agents==1.1.0b4 -U
! pip install azure-identity -U
! pip install mcp==1.11.0 -U

## استيراد المكتبات المطلوبة

قم باستيراد وحدات Python الضرورية:
- **os, time**: مكتبات Python القياسية للمتغيرات البيئية والتأخيرات
- **AIProjectClient**: العميل الرئيسي لمشاريع Azure AI
- **DefaultAzureCredential**: المصادقة بدون مفتاح لخدمات Azure
- **MCP-related classes**: لإنشاء وإدارة أدوات MCP ومعالجة الموافقات


In [ ]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval


## إعدادات خادم MCP

قم بإعداد تكوين خادم MCP باستخدام متغيرات البيئة مع قيم افتراضية احتياطية:
- **MCP_SERVER_URL**: عنوان URL لخادم MCP (القيمة الافتراضية هي واجهة برمجة تطبيقات Microsoft Learn)
- **MCP_SERVER_LABEL**: تسمية لتحديد خادم MCP (القيمة الافتراضية هي "mslearn")

تتيح هذه الطريقة تكوينًا مرنًا عبر بيئات مختلفة.


In [ ]:
mcp_server_url = os.environ.get("MCP_SERVER_URL", "https://learn.microsoft.com/api/mcp")
mcp_server_label = os.environ.get("MCP_SERVER_LABEL", "mslearn")

## إنشاء عميل مشروع Azure AI (مصادقة بدون مفتاح)

قم بتهيئة عميل مشروع Azure AI باستخدام **المصادقة بدون مفتاح**:
- **النقطة النهائية**: عنوان URL لنقطة النهاية الخاصة بمشروع Azure AI Foundry
- **بيانات الاعتماد**: يستخدم `DefaultAzureCredential()` للمصادقة الآمنة بدون مفتاح
- **لا حاجة لمفاتيح API**: يكتشف تلقائيًا ويستخدم أفضل بيانات اعتماد متاحة

**تدفق المصادقة:**
1. يتحقق من الهوية المُدارة (في بيئات Azure)
2. ينتقل إلى بيانات اعتماد Azure CLI (للتطوير المحلي)
3. يستخدم مصادر بيانات الاعتماد الأخرى حسب الحاجة

هذا النهج يلغي الحاجة إلى إدارة مفاتيح API أو سلاسل الاتصال في الكود الخاص بك.


In [ ]:
project_client = AIProjectClient(
    endpoint="Your Azure AI Foundry Endpoint",
    credential=DefaultAzureCredential(),
)

## إنشاء تعريف أداة MCP

قم بإنشاء أداة MCP تتصل بخادم Microsoft Learn MCP:
- **server_label**: معرف لخادم MCP
- **server_url**: نقطة النهاية لخادم MCP
- **allowed_tools**: قائمة اختيارية لتقييد الأدوات التي يمكن استخدامها (القائمة الفارغة تسمح بجميع الأدوات)

ستُمكّن هذه الأداة الوكيل من الوصول إلى وثائق وموارد Microsoft Learn.


In [ ]:
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=[],  # Optional: specify allowed tools
)


## إنشاء وكيل وتنفيذ محادثة (عملية بدون مفاتيح)

يشرح هذا القسم الشامل عملية **الوكيل بدون مفاتيح** بالكامل:

1. **إنشاء وكيل ذكاء اصطناعي**: إعداد وكيل باستخدام نموذج GPT-4.1 nano وأدوات MCP  
2. **إنشاء سلسلة محادثة**: إنشاء سلسلة محادثة للتواصل  
3. **إرسال رسالة**: اسأل الوكيل عن الفروقات بين Azure OpenAI و OpenAI  
4. **التعامل مع موافقات الأدوات**: الموافقة تلقائيًا على استدعاءات أدوات MCP عند الحاجة  
5. **مراقبة التنفيذ**: تتبع تقدم الوكيل والتعامل مع أي إجراءات مطلوبة  
6. **عرض النتائج**: عرض تفاصيل المحادثة واستخدام الأدوات  

**ميزات بدون مفاتيح:**
- ✅ **لا توجد أسرار مبرمجة مسبقًا** - يتم التعامل مع جميع المصادقات بواسطة هوية Azure  
- ✅ **آمن بشكل افتراضي** - يستخدم التحكم في الوصول بناءً على الأدوار  
- ✅ **نشر مبسط** - لا حاجة لإدارة بيانات الاعتماد  
- ✅ **ملائم للتدقيق** - يتم تتبع جميع الوصول من خلال هوية Azure  

سيستخدم الوكيل أدوات MCP للوصول إلى موارد Microsoft Learn بأمان كامل ودون الحاجة لإدارة مفاتيح API.


In [ ]:
with project_client:
    agents_client = project_client.agents

    # Create a new agent with keyless authentication
    # NOTE: To reuse existing agent, fetch it with get_agent(agent_id)
    agent = agents_client.create_agent(
        model="Your Azure OpenAI Model Deployment Name",
        name="my-mcp-agent",
        instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
        tools=mcp_tool.definitions,
    )
    print(f"Created agent, ID: {agent.id}")
    print(f"MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

    # Create thread for communication
    thread = agents_client.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content="What's difference between Azure OpenAI and OpenAI?",
    )
    print(f"Created message, ID: {message.id}")

    # KEYLESS APPROACH: Handle tool approvals without hardcoded secrets
    
    # Option 1: Completely keyless (recommended for Azure identity-enabled MCP servers)
    # run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    
    # Option 2: With minimal headers (if MCP server requires specific headers)
    # For demonstration purposes, using a placeholder header
    mcp_tool.update_headers("SuperSecret", "123456")  # Replace with actual auth if needed
    
    # Set approval mode - uncomment next line to disable approval requirement completely
    # mcp_tool.set_approval_mode("never")  # Fully automated, no approval needed
    
    run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    try:
                        print(f"Approving tool call: {tool_call}")
                        
                        # KEYLESS APPROVAL OPTIONS:
                        
                        # Option 1: No headers (fully keyless)
                        # tool_approvals.append(
                        #     ToolApproval(
                        #         tool_call_id=tool_call.id,
                        #         approve=True,
                        #         headers={}  # No headers needed for keyless
                        #     )
                        # )
                        
                        # Option 2: With headers (if MCP server requires them)
                        tool_approvals.append(
                            ToolApproval(
                                tool_call_id=tool_call.id,
                                approve=True,
                                headers=mcp_tool.headers,  # Uses configured headers if needed
                            )
                        )
                    except Exception as e:
                        print(f"Error approving tool_call {tool_call.id}: {e}")

            print(f"tool_approvals: {tool_approvals}")
            if tool_approvals:
                agents_client.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
                )

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Display run steps and tool calls
    run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)

    # Loop through each step
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")

        # Check if there are tool calls in the step details
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  MCP Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

        print()  # add an extra newline between steps

    # Fetch and log all messages
    messages = agents_client.messages.list(thread_id=thread.id)
    print("\nConversation:")
    print("-" * 50)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role.upper()}: {last_text.text.value}")
            print("-" * 50)

    # Example of dynamic tool management (keyless)
    print(f"\nDemonstrating keyless dynamic tool management:")
    print(f"Current allowed tools: {mcp_tool.allowed_tools}")
    print("✅ All operations completed using keyless authentication!")


---

**إخلاء المسؤولية**:  
تم ترجمة هذا المستند باستخدام خدمة الترجمة بالذكاء الاصطناعي [Co-op Translator](https://github.com/Azure/co-op-translator). بينما نسعى لتحقيق الدقة، يرجى العلم أن الترجمات الآلية قد تحتوي على أخطاء أو معلومات غير دقيقة. يجب اعتبار المستند الأصلي بلغته الأصلية المصدر الرسمي. للحصول على معلومات حاسمة، يُوصى بالاستعانة بترجمة بشرية احترافية. نحن غير مسؤولين عن أي سوء فهم أو تفسيرات خاطئة تنشأ عن استخدام هذه الترجمة.
